In [8]:
import os
import pandas as pd

def test(data):
    name =[x for x in globals() if globals()[x] is data][0]
    return name
def year_specific_DF(df_4m_Excel):
    processed_DF = df_4m_Excel
    processed_DF.fillna(0, inplace=True)
    processed_DF.columns = processed_DF.iloc[4]
    processed_DF = processed_DF.iloc[6:-3]
    if(test(df_4m_Excel) == 'df_2017_18_Apr2Mar'):
        processed_DF = processed_DF.iloc[:, [1,29,46,50,127,128,129,138,165,69,2]]
    else:
        processed_DF = processed_DF.iloc[:, [1,56,90,98,252,254,256,274,328,136,2]]
    processed_DF.iloc[ : , 10 ] = processed_DF.iloc[ : , 10 ].astype(int)
    processed_DF["MMR"] = (processed_DF["Total no. maternal deaths"].astype(str).str.replace(",","").astype(float) / processed_DF["Total number of pregnant women Registered for ANC"] * 1000).round(2)
    processed_DF.drop(processed_DF.columns[[9, 10]], axis = 1, inplace = True)
    processed_DF.rename(columns={ processed_DF.columns[0]: "Indicators" }, inplace = True)
    processed_DF.reset_index(inplace=True, drop = True)
    return processed_DF

final = pd.DataFrame(columns=['State','State ID','District','District ID','Year','ABY','Institutional Deliveries','IMR','Low Birth Weight','Polio Vaccine Coverage',
                                  'BCG Vaccine Coverage','Hepatitis Vaccine Coverage','Measles MMR Vaccine Coverage','MR Booster Coverage','MMR'])
initialRoot_Apr2Mar = r'C:\Users\Lenovo\Desktop\Data Preparation for Project\HMIS data\2017-2018\2017-2018\MonthUpToMarch'
x = 1

for path in os.listdir(initialRoot_Apr2Mar):
    df_2017_18_Apr2Mar = pd.read_excel(open(os.path.join(initialRoot_Apr2Mar, path, 'ALL.xlsx'), 'rb'), sheet_name=0)
    df_2018_19_Apr2Mar = pd.read_excel(open(os.path.join(initialRoot_Apr2Mar.replace('2017-2018', '2018-2019'), path, 'ALL.xlsx'), 'rb'), sheet_name=0)
    df_2019_20_Apr2Mar = pd.read_excel(open(os.path.join(initialRoot_Apr2Mar.replace('2017-2018', '2019-2020'), path, 'ALL.xlsx'), 'rb'), sheet_name=0)
    df_2017 = year_specific_DF(df_2017_18_Apr2Mar)
    df_2018 = year_specific_DF(df_2018_19_Apr2Mar)
    df_2019 = year_specific_DF(df_2019_20_Apr2Mar)
    
    final_local = pd.DataFrame(columns=['State','State ID','District','District ID','Year','ABY','Institutional Deliveries','IMR','Low Birth Weight','Polio Vaccine Coverage',
                                  'BCG Vaccine Coverage','Hepatitis Vaccine Coverage','Measles MMR Vaccine Coverage','MR Booster Coverage','MMR'])

    for i, row in df_2017.iterrows():
        j = i + i * 2
    #   List contains State ID and ABY Status
    #   Create list for 2017 and add it as a row in final dataframe
        list1 = []
        list1.insert(0, df_2017.iloc[0]['Indicators'].replace('_',''))
        list1.insert(1, x)
        list1.insert(2, df_2017.iloc[i]['Indicators'])
        list1.insert(3, str(i))
        list1.insert(4, '2017')
        list1.insert(5, '0')
        final_local.loc[j] = list1 + list(df_2017.iloc[i, 1:])
    #   Create list for 2018 and add it as a row in final dataframe
        list1.pop(4)
        list1.insert(4, '2018')
        list1.pop(5)
        if((df_2017.iloc[0]['Indicators'].replace('_','') == "Delhi") or (df_2017.iloc[0]['Indicators'].replace('_','') == "Puducherry") or (df_2017.iloc[0]['Indicators'].replace('_','') == "Telangana") or (df_2017.iloc[0]['Indicators'].replace('_','') == "West Bengal")):
            list1.insert(5, '0')
        else:
            list1.insert(5, '1')
        final_local.loc[j + 1] = list1 + list(df_2018.iloc[i, 1:])
    #   Create list for 2018 and add it as a row in final dataframe Note: data for 2019 is not available
        list1.pop(4)
        list1.insert(4, '2019')
        list1.pop(5)
        if((df_2017.iloc[0]['Indicators'].replace('_','') == "Delhi") or (df_2017.iloc[0]['Indicators'].replace('_','') == "Puducherry") or (df_2017.iloc[0]['Indicators'].replace('_','') == "Telangana") or (df_2017.iloc[0]['Indicators'].replace('_','') == "West Bengal")):
            list1.insert(5, '0')
        else:
            list1.insert(5, '1')
        final_local.loc[j + 2] = list1 + list(df_2019.iloc[i, 1:])
    final_local = final_local.iloc[3:]
    final_local = final_local.reset_index(drop=True)
    final = final.append(final_local, ignore_index = True)
    x = x+1

# Add doctor data in a list and make it a column of the final dataframe
doctors_dataframe = pd.read_excel(open(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\DOctor\12345.xlsx', 'rb'), sheet_name='Sheet1')
doctorList = []
for i, row in doctors_dataframe.iterrows():
    for j in range(0,doctors_dataframe.iloc[i]['No. of Districts']):
        a = []
        a.append(doctors_dataframe.iloc[i]['avg2017'])
        a.append(doctors_dataframe.iloc[i]['avg2018'])
        a.append(doctors_dataframe.iloc[i]['avg2019'])
        doctorList.extend(a) 
final["Doctor"] = doctorList

#Add Health Infrastructure data

health_Data = pd.DataFrame(columns=['District', 'Sub Centres','PHCs','CHCs','Sub Divisional Hospitals','District Hospitals', 'Total Health Centres'])
dir_2017 = r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Health Infra\2017'
dir_2018 = r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Health Infra\2018'
dir_2019 = r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Health Infra\2019'

for path in os.listdir(dir_2017):
    myDf_2017 = pd.read_excel(open(os.path.join(dir_2017, path), 'rb'), sheet_name=0)
    myDf_2018 = pd.read_excel(open(os.path.join(dir_2018, path), 'rb'), sheet_name=0)
    myDf_2019 = pd.read_excel(open(os.path.join(dir_2019, path), 'rb'), sheet_name=0)
                            
    myDf_2017 = myDf_2017[['Monthwise Status of Data Reporting: State (Across Districts)', 'Unnamed: 1', 'Unnamed: 7']].iloc[16:-2]
    myDf_2017.fillna(method='ffill', inplace=True)
    
    myDf_2018 = myDf_2018[['Monthwise Status of Data Reporting: State (Across Districts)', 'Unnamed: 1', 'Unnamed: 7']].iloc[16:-2]
    myDf_2018.fillna(method='ffill', inplace=True)
    
    myDf_2019 = myDf_2019[['Monthwise Status of Data Reporting: State (Across Districts)', 'Unnamed: 1', 'Unnamed: 7']].iloc[16:-2]
    myDf_2019.fillna(method='ffill', inplace=True)

    state_Data = pd.DataFrame(columns=['District', 'Sub Centres','PHCs','CHCs','Sub Divisional Hospitals','District Hospitals', 'Total Health Centres'])
    for i in range(0, int(round(len(myDf_2017.index)/6))):
        r = i + i * 2
        myList_2017 = []
        myList_2018 = []
        myList_2019 = []
        myList_2017.append(myDf_2017.iloc[6*i]['Monthwise Status of Data Reporting: State (Across Districts)'])
        myList_2018.append(myDf_2018.iloc[6*i]['Monthwise Status of Data Reporting: State (Across Districts)'])
        myList_2019.append(myDf_2019.iloc[6*i]['Monthwise Status of Data Reporting: State (Across Districts)'])
        for j in range(6*i, 6*i+6):
            myList_2017.append(myDf_2017.iloc[j]['Unnamed: 7'])
            myList_2018.append(myDf_2018.iloc[j]['Unnamed: 7'])
            myList_2019.append(myDf_2019.iloc[j]['Unnamed: 7'])
        state_Data.loc[r] = myList_2017
        state_Data.loc[r+1] = myList_2018
        state_Data.loc[r+2] = myList_2019
    health_Data = health_Data.append(state_Data, ignore_index = True)
# Line below is to delete the district column of the health infra dataframe to avoid duplicacy....it was added to verify
# the correctness of data
health_Data = health_Data[health_Data.columns[1:]] 
final = pd.concat([final, health_Data], axis=1, sort=False)


# Add GDP data in a list and make it a column of the final dataframe
gdp_dataframe = pd.read_excel(open(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\GDP\GDPCurrPrice.xlsx', 'rb'), sheet_name='GDPatCurrPrice')
gdpList = []
for i, row in gdp_dataframe.iterrows():
    for j in range(0,gdp_dataframe.iloc[i]['No. of Districts']):
        a = []
        a.append(int(gdp_dataframe.iloc[i]['gdp_2017']))
        a.append(int(gdp_dataframe.iloc[i]['gdp_2018']))
        a.append(int(gdp_dataframe.iloc[i]['gdp_2019']))
        gdpList.extend(a)  
final["GDP_PC"] = gdpList    
final.to_csv(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Output Files\processedData.csv')
nonABY = final.loc[final['State'].isin(['Puducherry', 'West Bengal', 'Telangana', 'Delhi'])]
nonABY.to_csv(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Output Files\processedData_nonABY.csv')
ABY = final.loc[~final['State'].isin(['Puducherry', 'West Bengal', 'Telangana', 'Delhi'])]
ABY.to_csv(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Output Files\processedData_ABY.csv')
print("done")
final

done


,State,State ID,District,District ID,Year,ABY,Institutional Deliveries,IMR,Low Birth Weight,Polio Vaccine Coverage,...,MR Booster Coverage,MMR,Doctor,Sub Centres,PHCs,CHCs,Sub Divisional Hospitals,District Hospitals,Total Health Centres,GDP_PC
0,A & N Islands,1,Nicobar,1,2017,0,93.9,99.1,15.7,80.7,...,13,0.00,15,41,4,1,1,1,47,159664
1,A & N Islands,1,Nicobar,1,2018,1,92.6,97.6,23.5,83.7,...,704,0.00,23,41,4,1,1,1,47,167647
2,A & N Islands,1,Nicobar,1,2019,1,96.1,97.3,14.8,87.4,...,284,2.06,35,41,4,1,1,1,47,176029
3,A & N Islands,1,North and Middle Andaman,2,2017,0,93.2,97.8,19.7,73.4,...,20,4.75,15,44,8,2,2,1,55,159664
4,A & N Islands,1,North and Middle Andaman,2,2018,1,92.7,97.9,14.5,89.9,...,1042,0.81,23,44,8,2,2,1,55,167647
5,A & N Islands,1,North and Middle Andaman,2,2019,1,94.1,98.6,19.9,98.8,...,922,2.97,35,44,8,2,2,1,55,176029
6,A & N Islands,1,South Andaman,3,2017,0,99.3,98.8,16,76.2,...,67,1.53,15,39,15,1,1,1,56,159664
7,A & N Islands,1,South Andaman,3,2018,1,99.3,98.3,15.8,71.9,...,2208,1.90,23,39,15,1,1,1,56,167647
8,A & N Islands,1,South Andaman,3,2019,1,99.2,98.6,14.9,88.5,...,2123,0.39,35,39,15,1,1,1,56,176029
9,Andhra Pradesh,2,Anantapur,1,2017,0,99.4,98.9,9.2,82.1,...,24837,0.73,189,586,121,16,3,2,728,139679


In [26]:
import os
import pandas as pd

def test(data):
    name =[x for x in globals() if globals()[x] is data][0]
    return name
def year_specific_DF(df_4m_Excel):
    processed_DF = df_4m_Excel
    processed_DF.fillna(0, inplace=True)
    processed_DF.columns = processed_DF.iloc[4]
    processed_DF = processed_DF.iloc[6:-3]
    if(test(df_4m_Excel) == 'df_2017_18_Apr2Mar'):
        processed_DF = processed_DF.iloc[:, [1,29,43,46,50,127,128,129,138,165,190,69,2]]
    else:
        processed_DF = processed_DF.iloc[:, [1,56,84,90,98,252,254,256,274,328,378,136,2]]
    processed_DF.iloc[ : , 2 ] = processed_DF.iloc[ : , 2 ].astype(int)
    processed_DF["MMR"] = (processed_DF["Total no. maternal deaths"].astype(str).str.replace(",","").astype(float) / processed_DF["Total Number of reported live births"] * 100000).round(2)
    processed_DF["IMR"] = (processed_DF["Total Number of Infant Deaths reported"].astype(str).str.replace(",","").astype(float) / processed_DF["Total Number of reported live births"] * 1000).round(2)
    processed_DF.rename(columns={ processed_DF.columns[0]: "Indicators" }, inplace = True)
    processed_DF.reset_index(inplace=True, drop = True)
    return processed_DF

final = pd.DataFrame(columns=['State','State ID','District','District ID','Year','ABY','Year2017','Year2018','Year2019','2018andABY','2019andABY','% Institutional deliveries to Total Reported Deliveries','Total Number of reported live births','% live birth to Reported Birth','% Newborns having weight less than 2.5 kg to Newborns weighed at birth','% Newborns given OPV0 at birth to Reported live birth',
                                  '% Newborns given BCG to Reported live birth','% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth','% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births','No of children 16-24 months age given Measles Rubella Vaccine 2nd dose',
                                  'Total Number of Infant Deaths reported','Total no. maternal deaths','Total number of pregnant women Registered for ANC','MMR','IMR'])
initialRoot_Apr2Mar = r'C:\Users\Lenovo\Desktop\Data Preparation for Project\HMIS data\2017-2018\2017-2018\MonthUpToMarch'
x = 1

for path in os.listdir(initialRoot_Apr2Mar):
    df_2017_18_Apr2Mar = pd.read_excel(open(os.path.join(initialRoot_Apr2Mar, path, 'ALL.xlsx'), 'rb'), sheet_name=0)
    df_2018_19_Apr2Mar = pd.read_excel(open(os.path.join(initialRoot_Apr2Mar.replace('2017-2018', '2018-2019'), path, 'ALL.xlsx'), 'rb'), sheet_name=0)
    df_2019_20_Apr2Mar = pd.read_excel(open(os.path.join(initialRoot_Apr2Mar.replace('2017-2018', '2019-2020'), path, 'ALL.xlsx'), 'rb'), sheet_name=0)
    df_2017 = year_specific_DF(df_2017_18_Apr2Mar)
    df_2018 = year_specific_DF(df_2018_19_Apr2Mar)
    df_2019 = year_specific_DF(df_2019_20_Apr2Mar)
    
    final_local = pd.DataFrame(columns=['State','State ID','District','District ID','Year','ABY','Year2017','Year2018','Year2019','2018andABY','2019andABY','% Institutional deliveries to Total Reported Deliveries','Total Number of reported live births','% live birth to Reported Birth','% Newborns having weight less than 2.5 kg to Newborns weighed at birth','% Newborns given OPV0 at birth to Reported live birth',
                                  '% Newborns given BCG to Reported live birth','% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth','% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births','No of children 16-24 months age given Measles Rubella Vaccine 2nd dose',
                                  'Total Number of Infant Deaths reported','Total no. maternal deaths','Total number of pregnant women Registered for ANC','MMR','IMR'])

    for i, row in df_2017.iterrows():
        j = i + i * 2
    #   List contains State ID and ABY Status
    #   Create list for 2017 and add it as a row in final dataframe
        list1 = []
        list1.insert(0, df_2017.iloc[0]['Indicators'].replace('_',''))
        list1.insert(1, x)
        list1.insert(2, df_2017.iloc[i]['Indicators'])
        list1.insert(3, str(i))
        list1.insert(4, '2017')
        list1.insert(5, '0')
        list1.insert(6, '1')
        list1.insert(7, '0')
        list1.insert(8, '0')
        list1.insert(9, int(list1[5]) * int(list1[7]))
        list1.insert(10, int(list1[5]) * int(list1[8]))
        final_local.loc[j] = list1 + list(df_2017.iloc[i, 1:])
    #   Create list for 2018 and add it as a row in final dataframe
        list1.pop(4)
        list1.insert(4, '2018')
        list1.pop(5)
        if((df_2017.iloc[0]['Indicators'].replace('_','') == "Delhi") or (df_2017.iloc[0]['Indicators'].replace('_','') == "Puducherry") or (df_2017.iloc[0]['Indicators'].replace('_','') == "Telangana") or (df_2017.iloc[0]['Indicators'].replace('_','') == "West Bengal")):
            list1.insert(5, '0')
        else:
            list1.insert(5, '1')
        list1.pop(6)
        list1.insert(6, '0')
        list1.pop(7)
        list1.insert(7, '1')
        list1.pop(8)
        list1.insert(8, '0')
        list1.pop(9)
        list1.insert(9, int(list1[5]) * int(list1[7]))
        list1.pop(10)
        list1.insert(10, int(list1[5]) * int(list1[8]))
        final_local.loc[j + 1] = list1 + list(df_2018.iloc[i, 1:])
    #   Create list for 2018 and add it as a row in final dataframe Note: data for 2019 is not available
        list1.pop(4)
        list1.insert(4, '2019')
        list1.pop(5)
        if((df_2017.iloc[0]['Indicators'].replace('_','') == "Delhi") or (df_2017.iloc[0]['Indicators'].replace('_','') == "Puducherry") or (df_2017.iloc[0]['Indicators'].replace('_','') == "Telangana") or (df_2017.iloc[0]['Indicators'].replace('_','') == "West Bengal")):
            list1.insert(5, '0')
        else:
            list1.insert(5, '1')
        list1.pop(6)
        list1.insert(6, '0')
        list1.pop(7)
        list1.insert(7, '0')
        list1.pop(8)
        list1.insert(8, '1')
        list1.pop(9)
        list1.insert(9, int(list1[5]) * int(list1[7]))
        list1.pop(10)
        list1.insert(10, int(list1[5]) * int(list1[8]))
        final_local.loc[j + 2] = list1 + list(df_2019.iloc[i, 1:])
    final_local = final_local.iloc[3:]
    final_local = final_local.reset_index(drop=True)
    final = final.append(final_local, ignore_index = True)
    x = x+1

# Add doctor data in a list and make it a column of the final dataframe
doctors_dataframe = pd.read_excel(open(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\DOctor\12345.xlsx', 'rb'), sheet_name='Sheet1')
doctorList = []
for i, row in doctors_dataframe.iterrows():
    for j in range(0,doctors_dataframe.iloc[i]['No. of Districts']):
        a = []
        a.append(doctors_dataframe.iloc[i]['avg2017'])
        a.append(doctors_dataframe.iloc[i]['avg2018'])
        a.append(doctors_dataframe.iloc[i]['avg2019'])
        doctorList.extend(a) 
final["Doctor"] = doctorList

#Add Health Infrastructure data

health_Data = pd.DataFrame(columns=['District', 'Sub Centres','PHCs','CHCs','Sub Divisional Hospitals','District Hospitals', 'Total Health Centres'])
dir_2017 = r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Health Infra\2017'
dir_2018 = r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Health Infra\2018'
dir_2019 = r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Health Infra\2019'

for path in os.listdir(dir_2017):
    myDf_2017 = pd.read_excel(open(os.path.join(dir_2017, path), 'rb'), sheet_name=0)
    myDf_2018 = pd.read_excel(open(os.path.join(dir_2018, path), 'rb'), sheet_name=0)
    myDf_2019 = pd.read_excel(open(os.path.join(dir_2019, path), 'rb'), sheet_name=0)
                            
    myDf_2017 = myDf_2017[['Monthwise Status of Data Reporting: State (Across Districts)', 'Unnamed: 1', 'Unnamed: 7']].iloc[16:-2]
    myDf_2017.fillna(method='ffill', inplace=True)
    
    myDf_2018 = myDf_2018[['Monthwise Status of Data Reporting: State (Across Districts)', 'Unnamed: 1', 'Unnamed: 7']].iloc[16:-2]
    myDf_2018.fillna(method='ffill', inplace=True)
    
    myDf_2019 = myDf_2019[['Monthwise Status of Data Reporting: State (Across Districts)', 'Unnamed: 1', 'Unnamed: 7']].iloc[16:-2]
    myDf_2019.fillna(method='ffill', inplace=True)

    state_Data = pd.DataFrame(columns=['District', 'Sub Centres','PHCs','CHCs','Sub Divisional Hospitals','District Hospitals', 'Total Health Centres'])
    for i in range(0, int(round(len(myDf_2017.index)/6))):
        r = i + i * 2
        myList_2017 = []
        myList_2018 = []
        myList_2019 = []
        myList_2017.append(myDf_2017.iloc[6*i]['Monthwise Status of Data Reporting: State (Across Districts)'])
        myList_2018.append(myDf_2018.iloc[6*i]['Monthwise Status of Data Reporting: State (Across Districts)'])
        myList_2019.append(myDf_2019.iloc[6*i]['Monthwise Status of Data Reporting: State (Across Districts)'])
        for j in range(6*i, 6*i+6):
            myList_2017.append(myDf_2017.iloc[j]['Unnamed: 7'])
            myList_2018.append(myDf_2018.iloc[j]['Unnamed: 7'])
            myList_2019.append(myDf_2019.iloc[j]['Unnamed: 7'])
        state_Data.loc[r] = myList_2017
        state_Data.loc[r+1] = myList_2018
        state_Data.loc[r+2] = myList_2019
    health_Data = health_Data.append(state_Data, ignore_index = True)
# Line below is to delete the district column of the health infra dataframe to avoid duplicacy....it was added to verify
# the correctness of data
health_Data = health_Data[health_Data.columns[1:]] 
final = pd.concat([final, health_Data], axis=1, sort=False)


# Add GDP data in a list and make it a column of the final dataframe
gdp_dataframe = pd.read_excel(open(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\GDP\GDPCurrPrice.xlsx', 'rb'), sheet_name='GDPatCurrPrice')
gdpList = []
for i, row in gdp_dataframe.iterrows():
    for j in range(0,gdp_dataframe.iloc[i]['No. of Districts']):
        a = []
        a.append(int(gdp_dataframe.iloc[i]['gdp_2017']))
        a.append(int(gdp_dataframe.iloc[i]['gdp_2018']))
        a.append(int(gdp_dataframe.iloc[i]['gdp_2019']))
        gdpList.extend(a)  
final["GDP_PC"] = gdpList 
for i in range(len(final)): 
    if (final.loc[i, 'GDP_PC'] < final['GDP_PC'].median()):
        final.loc[i,'Poor_GDP_PC'] = "1"
    else: 
        final.loc[i,'Poor_GDP_PC'] = "0"
final.to_csv(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Output Files\processedData.csv')
nonABY = final.loc[final['State'].isin(['Puducherry', 'West Bengal', 'Telangana', 'Delhi'])]
nonABY.to_csv(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Output Files\processedData_nonABY.csv')
ABY = final.loc[~final['State'].isin(['Puducherry', 'West Bengal', 'Telangana', 'Delhi'])]
ABY.to_csv(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Output Files\processedData_ABY.csv')
print("done")
final

done


,State,State ID,District,District ID,Year,ABY,Year2017,Year2018,Year2019,2018andABY,...,IMR,Doctor,Sub Centres,PHCs,CHCs,Sub Divisional Hospitals,District Hospitals,Total Health Centres,GDP_PC,Poor_GDP_PC
0,A & N Islands,1,Nicobar,1,2017,0,1,0,0,0,...,33.33,15,41,4,1,1,1,47,159664,0
1,A & N Islands,1,Nicobar,1,2018,1,0,1,0,1,...,7.43,23,41,4,1,1,1,47,167647,0
2,A & N Islands,1,Nicobar,1,2019,1,0,0,1,0,...,27.97,35,41,4,1,1,1,47,176029,0
3,A & N Islands,1,North and Middle Andaman,2,2017,0,1,0,0,0,...,10.39,15,44,8,2,2,1,55,159664,0
4,A & N Islands,1,North and Middle Andaman,2,2018,1,0,1,0,1,...,12.46,23,44,8,2,2,1,55,167647,0
5,A & N Islands,1,North and Middle Andaman,2,2019,1,0,0,1,0,...,24.04,35,44,8,2,2,1,55,176029,0
6,A & N Islands,1,South Andaman,3,2017,0,1,0,0,0,...,19.77,15,39,15,1,1,1,56,159664,0
7,A & N Islands,1,South Andaman,3,2018,1,0,1,0,1,...,13.91,23,39,15,1,1,1,56,167647,0
8,A & N Islands,1,South Andaman,3,2019,1,0,0,1,0,...,17.87,35,39,15,1,1,1,56,176029,0
9,Andhra Pradesh,2,Anantapur,1,2017,0,1,0,0,0,...,14.87,174,586,121,16,3,2,728,139679,0


In [27]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
final['GDP_PC'].median()

110605.0

In [28]:
# important but experimental
# x_al = final
# x_al[['State ID','District ID','Year','ABY','Year2017','Year2018','Year2019','2018andABY','2019andABY','% Institutional deliveries to Total Reported Deliveries','Total Number of reported live births','% live birth to Reported Birth','% Newborns having weight less than 2.5 kg to Newborns weighed at birth','% Newborns given OPV0 at birth to Reported live birth',
#                                   '% Newborns given BCG to Reported live birth','% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth','% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births','No of children 16-24 months age given Measles Rubella Vaccine 2nd dose',
#                                   'Total Number of Infant Deaths reported','Total no. maternal deaths','Total number of pregnant women Registered for ANC','MMR','IMR','Doctor','Sub Centres','PHCs','CHCs','Sub Divisional Hospitals','District Hospitals', 'Total Health Centres','GDP_PC']] = x_al[['State ID','District ID','Year','ABY','Year2017','Year2018','Year2019','2018andABY','2019andABY','% Institutional deliveries to Total Reported Deliveries','Total Number of reported live births','% live birth to Reported Birth','% Newborns having weight less than 2.5 kg to Newborns weighed at birth','% Newborns given OPV0 at birth to Reported live birth',
#                                   '% Newborns given BCG to Reported live birth','% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth','% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births','No of children 16-24 months age given Measles Rubella Vaccine 2nd dose',
#                                   'Total Number of Infant Deaths reported','Total no. maternal deaths','Total number of pregnant women Registered for ANC','MMR','IMR','Doctor','Sub Centres','PHCs','CHCs','Sub Divisional Hospitals','District Hospitals', 'Total Health Centres','GDP_PC']].astype('float64')
# x_al.drop(['State ID','District ID'], axis = 1)
# x_al = x_al.groupby(['State','Year']).mean().round(2)
# x_al = pd.concat([x_all1, x_all2], axis=1, sort=False)
# x_al

In [29]:
source_final = final
source_final[['State ID','District ID','Year','ABY','Year2017','Year2018','Year2019','2018andABY','2019andABY','% Institutional deliveries to Total Reported Deliveries','Total Number of reported live births','% live birth to Reported Birth','% Newborns having weight less than 2.5 kg to Newborns weighed at birth','% Newborns given OPV0 at birth to Reported live birth',
                                  '% Newborns given BCG to Reported live birth','% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth','% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births','No of children 16-24 months age given Measles Rubella Vaccine 2nd dose',
                                  'Total Number of Infant Deaths reported','Total no. maternal deaths','Total number of pregnant women Registered for ANC','MMR','IMR','Doctor','Sub Centres','PHCs','CHCs','Sub Divisional Hospitals','District Hospitals', 'Total Health Centres','GDP_PC','Poor_GDP_PC']] = source_final[['State ID','District ID','Year','ABY','Year2017','Year2018','Year2019','2018andABY','2019andABY','% Institutional deliveries to Total Reported Deliveries','Total Number of reported live births','% live birth to Reported Birth','% Newborns having weight less than 2.5 kg to Newborns weighed at birth','% Newborns given OPV0 at birth to Reported live birth',
                                  '% Newborns given BCG to Reported live birth','% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth','% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births','No of children 16-24 months age given Measles Rubella Vaccine 2nd dose',
                                  'Total Number of Infant Deaths reported','Total no. maternal deaths','Total number of pregnant women Registered for ANC','MMR','IMR','Doctor','Sub Centres','PHCs','CHCs','Sub Divisional Hospitals','District Hospitals', 'Total Health Centres','GDP_PC','Poor_GDP_PC']].astype('float64')

x_all1 = source_final.iloc[:, [0,4,5,6,7,8,9,10,11,13,14,15,16,17,18,23,24,32,33]]
x_all2 = source_final.iloc[:, [0,4,12,19,20,21,22,25,26,27,28,29,30,31]]
x_all1 = x_all1.groupby(['State','Year']).mean().round(2)
x_all2 = x_all2.groupby(['State','Year']).sum().round(2)
x_all = pd.concat([x_all1, x_all2], axis=1, sort=False)
x_all.to_csv(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Output Files\All State.csv')
x_all

ABY  Year2017  Year2018  Year2019  2018andABY  \
State                Year                                                    
A & N Islands        2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Andhra Pradesh       2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Arunachal Pradesh    2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Assam                2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Bihar                2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Chandigarh           2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Chhattisgarh         2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Dadra & Nagar Haveli 2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Daman & Diu          2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Delhi                2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  0.0       0.0       1.0       0.0         0.0   
                     2019.0  0.0       0.0       0.0       1.0         0.0   
Goa                  2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Gujarat              2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Haryana              2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Himachal Pradesh     2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Jammu & Kashmir      2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Jharkhand            2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Karnataka            2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.

In [35]:
all_india1 = x_all1
all_india2 = x_all2
all_india1 = all_india1.groupby(['Year']).mean().round(2)
all_india2 = all_india2.groupby(['Year']).sum().round(2)
all_india = pd.concat([all_india1, all_india2], axis=1, sort=False)
all_india['ABY'] = ['0.0', '1.0', '1.0']
all_india['2018andABY'] = ['0.0', '1.0', '0.0']
all_india['2019andABY'] = ['0.0', '0.0', '1.0']
all_india['Poor_GDP_PC'] = ['1.0', '1.0', '0.0']
all_india.to_csv(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Output Files\All India.csv')
all_india

,ABY,Year2017,Year2018,Year2019,2018andABY,2019andABY,% Institutional deliveries to Total Reported Deliveries,% live birth to Reported Birth,% Newborns having weight less than 2.5 kg to Newborns weighed at birth,% Newborns given OPV0 at birth to Reported live birth,% Newborns given BCG to Reported live birth,% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth,% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births,MMR,IMR,GDP_PC,Poor_GDP_PC,Total Number of reported live births,No of children 16-24 months age given Measles Rubella Vaccine 2nd dose,Total Number of Infant Deaths reported,Total no. maternal deaths,Total number of pregnant women Registered for ANC,Doctor,Sub Centres,PHCs,CHCs,Sub Divisional Hospitals,District Hospitals,Total Health Centres
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017.0,0.0,1.0,0.0,0.0,0.0,0.0,91.79,98.71,11.63,87.67,110.12,74.84,116.43,108.12,14.50,149732.42,1.0,20784374.0,2518167.0,234952.0,21538.0,29002405.0,45574.0,162024.0,30910.0,12237.0,7217.0,1377.0,206966.0
2018.0,1.0,0.0,1.0,0.0,1.0,0.0,92.56,98.67,12.22,86.77,109.96,75.01,117.77,88.92,14.30,165591.89,1.0,20889154.0,9910535.0,245135.0,20069.0,28787946.0,62802.0,162283.0,31181.0,13179.0,8216.0,1490.0,208675.0
2019.0,1.0,0.0,0.0,1.0,0.0,1.0,93.39,98.72,12.40,90.01,111.88,79.46,125.14,109.89,14.31,176610.86,0.0,19670913.0,19760312.0,225170.0,19059.0,28129307.0,107008.0,162317.0,31202.0,13291.0,8329.0,1491.0,208844.0


In [31]:
source_nonABY = nonABY
source_nonABY[['State ID','District ID','Year','ABY','Year2017','Year2018','Year2019','2018andABY','2019andABY','% Institutional deliveries to Total Reported Deliveries','Total Number of reported live births','% live birth to Reported Birth','% Newborns having weight less than 2.5 kg to Newborns weighed at birth','% Newborns given OPV0 at birth to Reported live birth',
                                  '% Newborns given BCG to Reported live birth','% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth','% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births','No of children 16-24 months age given Measles Rubella Vaccine 2nd dose',
                                  'Total Number of Infant Deaths reported','Total no. maternal deaths','Total number of pregnant women Registered for ANC','MMR','IMR','Doctor','Sub Centres','PHCs','CHCs','Sub Divisional Hospitals','District Hospitals', 'Total Health Centres','GDP_PC','Poor_GDP_PC']] = source_nonABY[['State ID','District ID','Year','ABY','Year2017','Year2018','Year2019','2018andABY','2019andABY','% Institutional deliveries to Total Reported Deliveries','Total Number of reported live births','% live birth to Reported Birth','% Newborns having weight less than 2.5 kg to Newborns weighed at birth','% Newborns given OPV0 at birth to Reported live birth',
                                  '% Newborns given BCG to Reported live birth','% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth','% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births','No of children 16-24 months age given Measles Rubella Vaccine 2nd dose',
                                  'Total Number of Infant Deaths reported','Total no. maternal deaths','Total number of pregnant women Registered for ANC','MMR','IMR','Doctor','Sub Centres','PHCs','CHCs','Sub Divisional Hospitals','District Hospitals', 'Total Health Centres','GDP_PC','Poor_GDP_PC']].astype('float64')
xd1 = source_nonABY.iloc[:, [0,4,5,6,7,8,9,10,11,13,14,15,16,17,18,23,24,32,33]]
xd2 = source_nonABY.iloc[:, [0,4,12,19,20,21,22,25,26,27,28,29,30,31]]
xd1 = xd1.groupby(['State','Year']).mean().round(2)
xd2 = xd2.groupby(['State','Year']).sum().round(2)
xd = pd.concat([xd1, xd2], axis=1, sort=False)
xd.to_csv(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Output Files\nonABYstate.csv')
xd

c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


ABY  Year2017  Year2018  Year2019  2018andABY  2019andABY  \
State       Year                                                                
Delhi       2017.0  0.0       1.0       0.0       0.0         0.0         0.0   
            2018.0  0.0       0.0       1.0       0.0         0.0         0.0   
            2019.0  0.0       0.0       0.0       1.0         0.0         0.0   
Puducherry  2017.0  0.0       1.0       0.0       0.0         0.0         0.0   
            2018.0  0.0       0.0       1.0       0.0         0.0         0.0   
            2019.0  0.0       0.0       0.0       1.0         0.0         0.0   
Telangana   2017.0  0.0       1.0       0.0       0.0         0.0         0.0   
            2018.0  0.0       0.0       1.0       0.0         0.0         0.0   
            2019.0  0.0       0.0       0.0       1.0         0.0         0.0   
West Bengal 2017.0  0.0       1.0       0.0       0.0         0.0         0.0   
            2018.0  0.0       0.0       1.0       0.0         0.0         0.0   
            2019.0  0.0       0.0       0.0       1.0         0.0         0.0   

                    % Institutional deliveries to Total Reported Deliveries  \
State       Year                                                              
Delhi       2017.0                                              92.39         
            2018.0                                              92.80         
            2019.0                                              93.23         
Puducherry  2017.0                                              99.98         
            2018.0                                              99.80         
            2019.0                                             100.00         
Telangana   2017.0                                              99.34         
            2018.0                                              99.77         
            2019.0                                              99.80         
West Bengal 2017.0                                              96.00         
            2018.0                                              98.09         
            2019.0                                              98.66         

                    % live birth to Reported Birth  \
State       Year                                     
Delhi       2017.0                           98.49   
            2018.0                           98.54   
            2019.0                           98.54   
Puducherry  2017.0                           99.35   
            2018.0                           99.48   
            2019.0                           99.45   
Telangana   2017.0                           99.46   
            2018.0                           99.55   
            2019.0                           99.27   
West Bengal 2017.0                           98.34   
            2018.0                           98.43   
            2019.0                           98.50   

                    % Newborns having weight less than 2.5 kg to Newborns weighed at birth  \
State       Year                                                                             
Delhi       2017.0                                              18.90                        
            2018.0                                              20.16                        
            2019.0                                              21.65                        
Puducherry  2017.0                                               8.50                        
            2018.0                                               8.90                        
            2019.0                                               8.30                        
Telangana   2017.0                                               6.68                        
            2018.0                                               7.09                        
            2019.0                                               8.15            

In [32]:
xs1 = xd1
xs2 = xd2
xs1 = xs1.groupby(['Year']).mean().round(2)
xs2 = xs2.groupby(['Year']).sum().round(2)
xs = pd.concat([xs1, xs2], axis=1, sort=False)
xs['ABY'] = ['0.0', '1.0', '1.0']
xs['2018andABY'] = ['0.0', '1.0', '0.0']
xs['2019andABY'] = ['0.0', '0.0', '1.0']
xs['Poor_GDP_PC'] = ['1.0', '1.0', '0.0']
xs.to_csv(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Output Files\All India nonABY.csv')
xs

,ABY,Year2017,Year2018,Year2019,2018andABY,2019andABY,% Institutional deliveries to Total Reported Deliveries,% live birth to Reported Birth,% Newborns having weight less than 2.5 kg to Newborns weighed at birth,% Newborns given OPV0 at birth to Reported live birth,% Newborns given BCG to Reported live birth,% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth,% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births,MMR,IMR,GDP_PC,Poor_GDP_PC,Total Number of reported live births,No of children 16-24 months age given Measles Rubella Vaccine 2nd dose,Total Number of Infant Deaths reported,Total no. maternal deaths,Total number of pregnant women Registered for ANC,Doctor,Sub Centres,PHCs,CHCs,Sub Divisional Hospitals,District Hospitals,Total Health Centres
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017.0,0.0,1.0,0.0,0.0,0.0,0.0,96.93,98.91,13.34,92.56,106.09,82.89,123.66,85.70,12.93,201693.00,1.0,2252996.0,232452.0,32113.0,2089.0,3907073.0,3456.0,16517.0,2779.0,1195.0,279.0,209.0,20153.0
2018.0,1.0,0.0,1.0,0.0,1.0,0.0,97.62,99.00,14.16,92.72,106.08,83.86,123.41,79.33,13.31,224991.75,1.0,2263358.0,1655145.0,33285.0,2122.0,3428708.0,4526.0,16541.0,2835.0,1177.0,269.0,206.0,20203.0
2019.0,1.0,0.0,0.0,1.0,0.0,1.0,97.92,98.94,14.96,98.21,110.19,91.20,129.11,81.30,12.02,241066.25,0.0,2033858.0,2151939.0,27258.0,1895.0,3101233.0,11598.0,16542.0,2836.0,1177.0,269.0,206.0,20205.0


In [33]:
source_ABY = ABY
source_ABY[['State ID','District ID','Year','ABY','Year2017','Year2018','Year2019','2018andABY','2019andABY','% Institutional deliveries to Total Reported Deliveries','Total Number of reported live births','% live birth to Reported Birth','% Newborns having weight less than 2.5 kg to Newborns weighed at birth','% Newborns given OPV0 at birth to Reported live birth',
                                  '% Newborns given BCG to Reported live birth','% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth','% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births','No of children 16-24 months age given Measles Rubella Vaccine 2nd dose',
                                  'Total Number of Infant Deaths reported','Total no. maternal deaths','Total number of pregnant women Registered for ANC','MMR','IMR','Doctor','Sub Centres','PHCs','CHCs','Sub Divisional Hospitals','District Hospitals', 'Total Health Centres','GDP_PC','Poor_GDP_PC']] = source_ABY[['State ID','District ID','Year','ABY','Year2017','Year2018','Year2019','2018andABY','2019andABY','% Institutional deliveries to Total Reported Deliveries','Total Number of reported live births','% live birth to Reported Birth','% Newborns having weight less than 2.5 kg to Newborns weighed at birth','% Newborns given OPV0 at birth to Reported live birth',
                                  '% Newborns given BCG to Reported live birth','% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth','% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births','No of children 16-24 months age given Measles Rubella Vaccine 2nd dose',
                                  'Total Number of Infant Deaths reported','Total no. maternal deaths','Total number of pregnant women Registered for ANC','MMR','IMR','Doctor','Sub Centres','PHCs','CHCs','Sub Divisional Hospitals','District Hospitals', 'Total Health Centres','GDP_PC','Poor_GDP_PC']].astype('float64')
xe1 = source_ABY.iloc[:, [0,4,5,6,7,8,9,10,11,13,14,15,16,17,18,23,24,32,33]]
xe2 = source_ABY.iloc[:, [0,4,12,19,20,21,22,25,26,27,28,29,30,31]]
xe1 = xe1.groupby(['State','Year']).mean().round(2)
xe2 = xe2.groupby(['State','Year']).sum().round(2)
xe = pd.concat([xe1, xe2], axis=1, sort=False)
xe.to_csv(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Output Files\ABYstate.csv')
xe

ABY  Year2017  Year2018  Year2019  2018andABY  \
State                Year                                                    
A & N Islands        2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Andhra Pradesh       2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Arunachal Pradesh    2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Assam                2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Bihar                2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Chandigarh           2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Chhattisgarh         2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Dadra & Nagar Haveli 2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Daman & Diu          2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Goa                  2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Gujarat              2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Haryana              2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Himachal Pradesh     2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Jammu & Kashmir      2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Jharkhand            2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Karnataka            2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.0       0.0         1.0   
                     2019.0  1.0       0.0       0.0       1.0         0.0   
Kerala               2017.0  0.0       1.0       0.0       0.0         0.0   
                     2018.0  1.0       0.0       1.

In [34]:
xt1 = xe1
xt2 = xe2
xt1 = xt1.groupby(['Year']).mean().round(2)
xt2 = xt2.groupby(['Year']).sum().round(2)
xt = pd.concat([xt1, xt2], axis=1, sort=False)
xt['ABY'] = ['0.0', '1.0', '1.0']
xt['2018andABY'] = ['0.0', '1.0', '0.0']
xt['2019andABY'] = ['0.0', '0.0', '1.0']
xt['Poor_GDP_PC'] = ['1.0', '1.0', '1.0']
xt.to_csv(r'C:\Users\Lenovo\Desktop\Data Preparation for Project\Output Files\All India ABY.csv')
xt

,ABY,Year2017,Year2018,Year2019,2018andABY,2019andABY,% Institutional deliveries to Total Reported Deliveries,% live birth to Reported Birth,% Newborns having weight less than 2.5 kg to Newborns weighed at birth,% Newborns given OPV0 at birth to Reported live birth,% Newborns given BCG to Reported live birth,% Newborns given Hep-B0(Birth Dose)at birth to Reported live birth,% Infants 0 to 11 months old who received Measles+ MR vaccine to reported live births,MMR,IMR,GDP_PC,Poor_GDP_PC,Total Number of reported live births,No of children 16-24 months age given Measles Rubella Vaccine 2nd dose,Total Number of Infant Deaths reported,Total no. maternal deaths,Total number of pregnant women Registered for ANC,Doctor,Sub Centres,PHCs,CHCs,Sub Divisional Hospitals,District Hospitals,Total Health Centres
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017.0,0.0,1.0,0.0,0.0,0.0,0.0,91.14,98.69,11.42,87.06,110.63,73.83,115.53,110.92,14.70,143237.34,1.0,18531378.0,2285715.0,202839.0,19449.0,25095332.0,42118.0,145507.0,28131.0,11042.0,6938.0,1168.0,186813.0
2018.0,1.0,0.0,1.0,0.0,1.0,0.0,91.92,98.63,11.98,86.03,110.45,73.91,117.06,90.12,14.42,158166.91,1.0,18625796.0,8255390.0,211850.0,17947.0,25359238.0,58276.0,145742.0,28346.0,12002.0,7947.0,1284.0,188472.0
2019.0,1.0,0.0,0.0,1.0,0.0,1.0,92.82,98.70,12.08,88.99,112.09,78.00,124.64,113.47,14.59,168553.94,1.0,17637055.0,17608373.0,197912.0,17164.0,25028074.0,95410.0,145775.0,28366.0,12114.0,8060.0,1285.0,188639.0
